# Systems of Linear Equations

Here we will see various functions for solving and manipulating systems of linear equations.

## Initialization

In [1]:
import numpy as np
import scipy.linalg as la

As we have seen, the main linear algebra routines are contained in `scipy.linalg`. Here we will focus on some of those from "Basics" and those related to the LU decomposition.

We begin by reminding ourselves of what is available in `scipy.linalg`.

In [2]:
la?

Type:        module
String form: <module 'scipy.linalg' from '/home/craig/opt/anaconda/envs/p250/lib/python3.8/site-packages/scipy/linalg/__init__.py'>
File:        ~/opt/anaconda/envs/p250/lib/python3.8/site-packages/scipy/linalg/__init__.py
Docstring:  
Linear algebra (:mod:`scipy.linalg`)

.. currentmodule:: scipy.linalg

Linear algebra functions.

.. eventually, we should replace the numpy.linalg HTML link with just `numpy.linalg`

.. seealso::

   `numpy.linalg <https://www.numpy.org/devdocs/reference/routines.linalg.html>`__
   for more linear algebra functions. Note that
   although `scipy.linalg` imports most of them, identically named
   functions from `scipy.linalg` may offer more or slightly differing
   functionality.


Basics

.. autosummary::
   :toctree: generated/

   inv - Find the inverse of a square matrix
   solve - Solve a linear system of equations
   solve_banded - Solve a banded linear system
   solveh_banded - Solve a Hermitian or symmetric banded system
   sol

Also as always we should look at the documentation for any and all functions that interest us.  Here we look at a few.  In the prelab and lab we will explore the use of `solve_banded`.

In [3]:
la.solve?

Signature:
la.solve(
    a,
    b,
    sym_pos=False,
    lower=False,
    overwrite_a=False,
    overwrite_b=False,
    debug=None,
    check_finite=True,
    assume_a='gen',
    transposed=False,
)
Docstring:
Solves the linear equation set ``a * x = b`` for the unknown ``x``
for square ``a`` matrix.

If the data matrix is known to be a particular type then supplying the
corresponding string to ``assume_a`` key chooses the dedicated solver.
The available options are

===================  ========
 generic matrix       'gen'
 symmetric            'sym'
 hermitian            'her'
 positive definite    'pos'
===================  ========

If omitted, ``'gen'`` is the default structure.

The datatype of the arrays define which solver is called regardless
of the values. In other words, even when the complex array entries have
precisely zero imaginary parts, the complex solver will be called based
on the data type of the array.

Parameters
----------
a : (N, N) array_like
    Square input 

In [4]:
la.lu?

Signature: la.lu(a, permute_l=False, overwrite_a=False, check_finite=True)
Docstring:
Compute pivoted LU decomposition of a matrix.

The decomposition is::

    A = P L U

where P is a permutation matrix, L lower triangular with unit
diagonal elements, and U upper triangular.

Parameters
----------
a : (M, N) array_like
    Array to decompose
permute_l : bool, optional
    Perform the multiplication P*L (Default: do not permute)
overwrite_a : bool, optional
    Whether to overwrite data in a (may improve performance)
check_finite : bool, optional
    Whether to check that the input matrix contains only finite numbers.
    Disabling may give a performance gain, but may result in problems
    (crashes, non-termination) if the inputs do contain infinities or NaNs.

Returns
-------
**(If permute_l == False)**

p : (M, M) ndarray
    Permutation matrix
l : (M, K) ndarray
    Lower triangular or trapezoidal matrix with unit diagonal.
    K = min(M, N)
u : (K, N) ndarray
    Upper triangular 

In [5]:
la.lu_factor?

Signature: la.lu_factor(a, overwrite_a=False, check_finite=True)
Docstring:
Compute pivoted LU decomposition of a matrix.

The decomposition is::

    A = P L U

where P is a permutation matrix, L lower triangular with unit
diagonal elements, and U upper triangular.

Parameters
----------
a : (M, M) array_like
    Matrix to decompose
overwrite_a : bool, optional
    Whether to overwrite data in A (may increase performance)
check_finite : bool, optional
    Whether to check that the input matrix contains only finite numbers.
    Disabling may give a performance gain, but may result in problems
    (crashes, non-termination) if the inputs do contain infinities or NaNs.

Returns
-------
lu : (N, N) ndarray
    Matrix containing U in its upper triangle, and L in its lower triangle.
    The unit diagonal elements of L are not stored.
piv : (N,) ndarray
    Pivot indices representing the permutation matrix P:
    row i of matrix was interchanged with row piv[i].

See also
--------
lu_solve :

In [6]:
la.lu_solve?

Signature: la.lu_solve(lu_and_piv, b, trans=0, overwrite_b=False, check_finite=True)
Docstring:
Solve an equation system, a x = b, given the LU factorization of a

Parameters
----------
(lu, piv)
    Factorization of the coefficient matrix a, as given by lu_factor
b : array
    Right-hand side
trans : {0, 1, 2}, optional
    Type of system to solve:

    =====  =========
    trans  system
    =====  =========
    0      a x   = b
    1      a^T x = b
    2      a^H x = b
    =====  =========
overwrite_b : bool, optional
    Whether to overwrite data in b (may increase performance)
check_finite : bool, optional
    Whether to check that the input matrices contain only finite numbers.
    Disabling may give a performance gain, but may result in problems
    (crashes, non-termination) if the inputs do contain infinities or NaNs.

Returns
-------
x : array
    Solution to the system

See also
--------
lu_factor : LU factorize a matrix

Examples
--------
>>> from scipy.linalg import lu_fact

Notice that there are a few functions related to the LU decomposition.  We will learn why this is the case below.

## Sample system

As a sample system we will consider the one studied in class,
$$ \begin{align}
10 x_1 - 7 x_2 + x_3 &= 8, \\
-3 x_1 + 2 x_2 + 6 x_3 &= 4, \\
5 x_1 - x_2 + 5 x_3 &= 6 .
\end{align} $$
As we saw we can write this in the form 
$$\mathsf{A}\vec x = \vec b, $$
where
$$ \mathsf{A} = \left( \begin{array}{rrr}
10 & -7 & 1 \\
-3 & 2 & 6 \\
5 & -1 & 5
\end{array} \right)
\quad\mathrm{and}\quad
\vec b = \left( \begin{array}{c} 8 \\ 4 \\ 6 \end{array} \right). $$

We can create these as arrays using `np.array`.  As noted last week,
we force the result to be a floating point array, not an integer array, by making any one of the entries a floating point number.  NumPy tries to use the "simplest" data type when it creates an array.  There are a few ways to force the type it chooses, this is one way.

In [7]:
A = np.array([ [10, -7, 1.], [-3, 2, 6], [5, -1, 5] ])
b = np.array([8., 4, 6])

### Solving the System

To solve this system we can, not surprisingly, use `solve`.

In [8]:
x = la.solve(A, b)
print(x)

[ 0. -1.  1.]


#### Verifying the Solution

To verify the solution we can directly evaluate $\mathsf{A} \vec x$ and compare it to $\vec b$.  Of course this only verifies that the `solve` function has worked correctly, it does not verify that we have entered $\mathsf{A}$ and $\vec b$ correctly.  There are two steps, one is to actually perform the multiplication and the second is to compare the $\mathsf{A} \vec x$ to $\vec b$.

Again, we learned about this last week.  We know we need to use `np.dot()` for the multiplication/

In [9]:
print('np.dot(A, x) =', np.dot(A,x))

np.dot(A, x) = [8. 4. 6.]


Notice that this returns a vector at that vector should be $\vec b$.  We can include a test to verify this.

In [10]:
assert(np.allclose(np.dot(A,x), b))

##### Infix Operator '`@'`

It turns out that there is now a terser way of doing these multiplications.  Starting with Python 3.5 there is shorthand for this, using `@`.  This is *only* available in Python 3.5 (and later).  While it is true that it will work for us, this is not portable to older versions of Python and it is not backported via the `__future__` module.  Thus, using it will mean writing code that only works in a specific version of Python.  If you are interested in details/rationale see [PEP 456](https://www.python.org/dev/peps/pep-0465/).

Should you use it? It is up to you.  Older versions of Python continue to exist, though it is much better to be using a newer version.  There have been many improvements.  As far as this course is concerned, whether you should use the `@` operator or the `np.dot` function will be left up to you.

To verify it works we do a test.  Notice it gives the same result as `np.dot`.

In [11]:
print('A @ x =', A @ x)

A @ x = [8. 4. 6.]


### Factoring the System

As discussed in class, the work required to solve the system of equations mostly involves manipulating the matrix, $\mathsf{A}$, and then performing the same manipulations on the right hand side of the equations, $\vec b$.  We could instead have many right hand sides (a two dimensional array with multiple columns, one for each set of values for which we want to find a solution).  This is handled by `solve`.  Alternatively, we can decompose $\mathsf{A}$ into pieces that encode the required manipulations using the LU decomposition.  The decomposition only needs to be performed once, it can then be applied whenever needed.  Finally, we saw that for numerical stability we should also pivot the matrix.  LU decomposition with pivoting is provided by `scipy.linalg.lu`.

In [12]:
(P, L, U) = la.lu(A)
print("Permutation matrix:\n", P)
print("Lower triangular matrix:\n", L)
print("Upper triangular matrix:\n", U)
# Verify that this is the decomposition of A
assert(np.allclose(P@L@U, A))

Permutation matrix:
 [[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
Lower triangular matrix:
 [[ 1.    0.    0.  ]
 [ 0.5   1.    0.  ]
 [-0.3  -0.04  1.  ]]
Upper triangular matrix:
 [[10.   -7.    1.  ]
 [ 0.    2.5   4.5 ]
 [ 0.    0.    6.48]]


By default we are given the permutation matrix and the lower and upper triangular matrices that, when recombined, produce $\mathsf{A}$.  This function is good when we want to see the decomposition in a form easy for us to read.  If we want to use the decomposition for solving systems of linear equations the information could be stored in a more efficient form for the computer's use.

Note that our choice of using generic arrays for storing matrices comes at a cost when we want to multiply many of them together.  Here we have used the `@` operator.  If we had used `np.dot` instead we would have needed to use nested calls to `np.dot`.  This can be slightly streamlined, but remains tedious.

Getting back to solving a system of equations, we can use `scipy.linalg.lu_factor` to decompose in a form more useful for the computer.

In [13]:
la.lu_factor(A)

(array([[10.  , -7.  ,  1.  ],
        [ 0.5 ,  2.5 ,  4.5 ],
        [-0.3 , -0.04,  6.48]]),
 array([0, 2, 2], dtype=int32))

Here L and U have been merged into a single matrix (notice in the form above that L had ones along the diagonal so they do not need to be stored) and the permutations are represented by a one dimensional array instead a matrix.  This is much more memory efficient, but is also much harder for us to read.  Even so, this can be used in `scipy.linalg.lu_solve`.  In fact, the `tuple` returned here is exactly what needs to be provided as the first argument to that function.

In [14]:
lufactors = la.lu_factor(A)
xlu = la.lu_solve(lufactors, b)
print("LU solution:", xlu)
# Verify that the solution is consistent with the previous one.
assert(np.allclose(xlu, x))

LU solution: [ 0. -1.  1.]


## Slightly Larger System

As an example of a slightly larger system and one where we want to find solutions with different right hand sides consider
$$\mathsf{A} = \left( \begin{array}{rrrr}
2 & -3 & 1 & 3 \\
1 & 4 & -3 & -3 \\
5 & 3 & -1 & -1 \\
3 & -6 & -3 & 1
\end{array} \right),
\quad
\vec b_1 = \left( \begin{array}{r}
-4 \\ 1 \\ 8 \\ -5
\end{array} \right),
\quad
\vec b_2 = \left( \begin{array}{r}
-10 \\ 0 \\ -3 \\ -24
\end{array} \right).
$$

In [15]:
A = np.array([ [2., -3, 1, 3], [1, 4, -3, -3], [5, 3, -1, -1], [3, -6, -3, 1]])
b1 = np.array([-4., 1, 8, -5])
b2 = np.array([-10., 0, -3, -24])

We can solve this directly using `solve`.

In [16]:
print("Solution for b1:", la.solve(A, b1))
print("Solution for b2:", la.solve(A, b2))

Solution for b1: [ 2. -1.  4. -5.]
Solution for b2: [-1.  1.  4. -3.]


Alternatively we can use an LU decomposition.  Notice the decomposition is only performed once.

In [17]:
lufactors = la.lu_factor(A)
print("LU solution for b1:", la.lu_solve(lufactors, b1))
print("LU solution for b2:", la.lu_solve(lufactors, b2))

LU solution for b1: [ 2. -1.  4. -5.]
LU solution for b2: [-1.  1.  4. -3.]


Finally, we could have solved for both right hand sides at once.  To do this we need to combine `b1` and `b2`.  We do this below.  In general this is the **wrong approach**.  In practice we should have created `b` as a two dimensional array and used array slicing to pull out the pieces we needed.  It is **almost never** a good idea to proceed as we have done here, that is, to create the individual pieces and then recombine them.  Even so, there are a few functions that allow us to combine arrays, you will have to discover them for yourself if you find a valid need for them.

In [18]:
b = np.vstack((b1, b2))
b

array([[ -4.,   1.,   8.,  -5.],
       [-10.,   0.,  -3., -24.]])

If we try to use this with `solve` it fails!

In [19]:
la.solve(A, b)

ValueError: Input b has to have same number of rows as input a

We get a `ValueError` and even told that `b` should have the same number of *rows* as `A`, not the same number of columns.  This is easy to fix, just take the transpose of `b`.  There is shorthand for doing this as shown below.

In [20]:
print("Original b:\n", b)
print("Transpose of b:\n", b.T)

Original b:
 [[ -4.   1.   8.  -5.]
 [-10.   0.  -3. -24.]]
Transpose of b:
 [[ -4. -10.]
 [  1.   0.]
 [  8.  -3.]
 [ -5. -24.]]


With this we find:

In [21]:
print("scipy.linalg.solve:\n", la.solve(A, b.T))
print("scipy.linalg.lu_solve:\n", la.lu_solve(lufactors, b.T))

scipy.linalg.solve:
 [[ 2. -1.]
 [-1.  1.]
 [ 4.  4.]
 [-5. -3.]]
scipy.linalg.lu_solve:
 [[ 2. -1.]
 [-1.  1.]
 [ 4.  4.]
 [-5. -3.]]


Each *column* of the result is the solution for each *column* in the array `b`.